In [15]:
# LP version – performance-optimised with precomputed shift mappings
# Now with: (a) soc_start baseline offset, (b) HiGHS-first solver with CBC fallback,
#           (c) constraints_mode = "auto" or "manual" for SoC/power limits.
# This took 14 mins for smoothed national existing using AEF.
# approx 8.5mins for national MEF optimisation

import pandas as pd
import numpy as np
from pyomo.environ import *
from pyomo.opt import SolverFactory
from collections import defaultdict

# === CONFIGURATION ===
data_path = r"C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data\Battery Emissions Optimisation Sheet.csv"
mode = "independent"      # "together" or "independent"
shift_window = 24         # allowed shift radius in 30-min periods

# Choose how to set SoC capacity and power caps: "auto" (rolling 2-day) or "manual"
constraints_mode = "manual"   # "auto" or "manual"

# Manual values (used iff constraints_mode == "manual")
manual_soc_half_range_mwh = 50     # SoC bounds will be [-X, +X]
manual_max_charge_power_mwh = 25    # per 30-minute period
manual_max_discharge_power_mwh = 25  # per 30-minute period

# Auto window settings
periods_per_day = 48
window_days = 2
window_periods = periods_per_day * window_days  # 96 for 2 days

# === LOAD & PREPROCESS DATA ===
df = pd.read_csv(data_path)
df['SettlementDate'] = df['SettlementDate'].astype(str)
df['SettlementPeriod'] = df['SettlementPeriod'].fillna(0).astype(int)
df['datetime'] = pd.to_datetime(df['SettlementDate'], dayfirst=True, errors='coerce') + \
                 pd.to_timedelta((df['SettlementPeriod'] - 1) * 30, unit='m')
df = df.dropna(subset=['datetime']).sort_values('datetime').reset_index(drop=True)

T = len(df)
charge = -df['BatteryChargeMWh'].fillna(0).to_numpy()   # make charge positive
discharge = df['BatteryDischargeMWh'].fillna(0).to_numpy()
price = df['EmissionFactor'].fillna(0).to_numpy()

# === Efficiency (data-driven; make manual if you prefer) ===
total_charge = charge.sum()
total_discharge = discharge.sum()
efficiency = total_discharge / total_charge if total_charge > 0 else 1.0

# === SoC capacity and power limits (AUTO vs MANUAL) ===
if constraints_mode.lower() == "auto":
    # Build SoC trace from original data
    soc_trace = efficiency * np.cumsum(charge) - np.cumsum(discharge)

    if len(soc_trace) >= window_periods:
        s = pd.Series(soc_trace)
        rolling_max = s.rolling(window=window_periods, min_periods=window_periods).max()
        rolling_min = s.rolling(window=window_periods, min_periods=window_periods).min()
        max_change = float((rolling_max - rolling_min).max())
    else:
        max_change = float(np.max(soc_trace) - np.min(soc_trace))

    soc_half_range = max_change / 2.0
    soc_min = -soc_half_range
    soc_max =  soc_half_range

    max_charge_power = float(charge.max())
    max_discharge_power = float(discharge.max())

elif constraints_mode.lower() == "manual":
    soc_half_range = float(manual_soc_half_range_mwh)
    soc_min = -soc_half_range
    soc_max =  soc_half_range
    max_charge_power = float(manual_max_charge_power_mwh)
    max_discharge_power = float(manual_max_discharge_power_mwh)

else:
    raise ValueError("constraints_mode must be 'auto' or 'manual'.")

# === BUILD MODEL ===
model = ConcreteModel()

# Index sets
model.T = RangeSet(0, T - 1)
model.S = RangeSet(0, T - 1)

# Shiftable pairs and fast lookup
valid_shifts = {(t, s) for t in range(T) for s in range(max(0, t - shift_window), min(T, t + shift_window + 1))}
model.shift_pairs = Set(dimen=2, initialize=valid_shifts)

shift_targets = defaultdict(list)   # s per t
shift_sources = defaultdict(list)   # t per s
for t, s in valid_shifts:
    shift_targets[t].append(s)
    shift_sources[s].append(t)

# Decision variables (LP relaxation)
model.x_charge = Var(model.shift_pairs, domain=UnitInterval)
model.x_discharge = Var(model.shift_pairs, domain=UnitInterval)

# Shifted energy at each destination s
def charge_shifted_rule(m, s):
    return sum(charge[t] * m.x_charge[t, s] for t in shift_sources[s])
model.charge_shifted = Expression(model.S, rule=charge_shifted_rule)

def discharge_shifted_rule(m, s):
    return sum(discharge[t] * m.x_discharge[t, s] for t in shift_sources[s])
model.discharge_shifted = Expression(model.S, rule=discharge_shifted_rule)

# Objective: maximise price-weighted net export
model.obj = Objective(
    expr=sum(price[s] * (model.discharge_shifted[s] - model.charge_shifted[s]) for s in model.S),
    sense=maximize
)

# Assignment constraints
if mode == "independent":
    model.charge_assignment = ConstraintList()
    model.discharge_assignment = ConstraintList()
    for t in range(T):
        model.charge_assignment.add(sum(model.x_charge[t, s] for s in shift_targets[t]) == 1.0)
        model.discharge_assignment.add(sum(model.x_discharge[t, s] for s in shift_targets[t]) == 1.0)
elif mode == "together":
    model.shared_assignment = ConstraintList()
    for t in range(T):
        for s in shift_targets[t]:
            model.shared_assignment.add(model.x_charge[t, s] == model.x_discharge[t, s])
        model.shared_assignment.add(sum(model.x_charge[t, s] for s in shift_targets[t]) == 1.0)
else:
    raise ValueError("mode must be 'independent' or 'together'.")

# Power caps per half-hour
model.charge_power_limit = ConstraintList()
model.discharge_power_limit = ConstraintList()
for s in range(T):
    model.charge_power_limit.add(model.charge_shifted[s] <= max_charge_power)
    model.discharge_power_limit.add(model.discharge_shifted[s] <= max_discharge_power)

# SoC tracking with free baseline offset (avoids infeasibility from symmetric bounds)
model.soc_start = Var(domain=Reals)
model.SoC = Var(model.S, domain=Reals)
model.soc_bounds = ConstraintList()
model.soc_def = ConstraintList()
for s in range(T):
    flow = efficiency * model.charge_shifted[s] - model.discharge_shifted[s]
    if s == 0:
        model.soc_def.add(model.SoC[s] == model.soc_start + flow)
    else:
        model.soc_def.add(model.SoC[s] == model.SoC[s - 1] + flow)
    model.soc_bounds.add(model.SoC[s] >= soc_min)
    model.soc_bounds.add(model.SoC[s] <= soc_max)

# === SOLVE (prefer HiGHS; fallback to CBC) ===
solver_used = None
results = None
last_err = None
for name in ("appsi_highs", "highs", "cbc"):
    try:
        opt = SolverFactory(name)
        if not opt.available():
            continue
        if name in ("appsi_highs", "highs"):
            try:
                opt.options.update({
                    "solver": "ipm",
                    "presolve": "on",
                    "threads": 0,      # all cores
                    "parallel": "on",
                    # Optional: slightly looser tolerances to reach crossover sooner
                    # "ipm_optimality_tolerance": 1e-6,
                    # "primal_feasibility_tolerance": 1e-7,
                    # "dual_feasibility_tolerance": 1e-7,
                })
            except Exception:
                pass
        solver_used = name
        results = opt.solve(model, tee=True)
        break
    except Exception as e:
        last_err = e
        continue

if results is None:
    raise RuntimeError(f"No LP solver available (tried appsi_highs, highs, cbc). Last error: {last_err}")

print("Solver used:", solver_used)
print("Status:", results.solver.status)
print("Termination:", results.solver.termination_condition)

if (results.solver.status != SolverStatus.ok) or (results.solver.termination_condition not in (TerminationCondition.optimal, TerminationCondition.locallyOptimal)):
    raise RuntimeError(f"Solver did not find optimal solution. Status={results.solver.status}, Termination={results.solver.termination_condition}")

# === POSTPROCESS ===
new_charge = np.zeros(T)
new_discharge = np.zeros(T)
for s in range(T):
    new_charge[s] = value(model.charge_shifted[s])
    new_discharge[s] = value(model.discharge_shifted[s])

df['OptimisedChargeMWh'] = -new_charge
df['OptimisedDischargeMWh'] = new_discharge

df.to_csv(data_path.replace('.csv', '_optimised_LP.csv'), index=False)
print("LP Relaxation Optimisation complete. Output saved.")


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
RUN!
LP   has 122640 rows; 1733281 cols; 5217360 nonzeros
Coefficient ranges:
  Matrix [1e+00, 2e+00]
  Cost   [2e-03, 5e+02]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 5e+01]
Presolving model
87599 rows, 1733280 cols, 5182268 nonzeros  0s
Dependent equations search running on 52559 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.05s (limit = 1000.00s)
87599 rows, 1733280 cols, 5182268 nonzeros  2s
Presolve : Reductions: rows 87599(-35041); columns 1733280(-1); elements 5182268(-35092)
Solving the presolved LP
IPX model has 87599 rows, 1733280 columns and 5182268 nonzeros
Input
    Number of variables:                                1733280
    Number of free variables:                           0
    Number of constraints:                              87599
    Number of equality constraints:                     52559
    Number of matrix entries